In [ ]:
import gradio as gr
from openai import OpenAI
import tomllib as tl

In [ ]:
# Define the path to the TOML file
toml_file_path = '../secrets.toml'

# Read the TOML file
with open(toml_file_path, 'rb') as f:
    toml_data = tl.load(f)

api_key = toml_data['OPEN_AI_API_KEY']

client = OpenAI(api_key=api_key)

In [20]:
system_message = "You are a text summariser. You will be given a text input and you need to summarise it."

In [ ]:
# Store conversation history
chat_history = []

def transcribe_audio(audio_file):
    if not audio_file:
        return "No audio file provided."
    try:
        with open(audio_file, "rb") as audio:
            transcription = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio
            )
        return transcription.text
    except Exception as e:
        return f"An error occurred during transcription: {str(e)}"

def chat_with_gpt(message, history):
    """Send the message to GPT and get a response"""
    # Convert history to the format expected by OpenAI API
    messages = []
    for human, ai in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": ai})
    
    # Add the current message
    messages.append({"role": "user", "content": message})
    
    try:
        # Call the OpenAI API to get a response
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        
        # Return the assistant's response
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {str(e)}"

def process_audio_and_respond(audio_file, chat_history):
    """Process the audio file and respond using GPT"""
    if not audio_file:
        return chat_history + [["Please upload an audio file.", "I'm waiting for an audio file to transcribe."]]
    
    # Transcribe the audio
    transcription = transcribe_audio(audio_file)
    
    # Add the transcription to the chat history as a user message
    chat_history.append([f"[Transcribed Audio]: {transcription}", ""])
    
    # Get response from GPT
    gpt_response = chat_with_gpt(transcription, chat_history[:-1])
    
    # Update the last assistant response
    chat_history[-1][1] = gpt_response
    
    return chat_history

def submit_text(text, chat_history):
    """Handle text input from the user"""
    # Add the user message to history
    chat_history.append([text, ""])
    
    # Get response from GPT
    gpt_response = chat_with_gpt(text, chat_history[:-1])
    
    # Update the last assistant response
    chat_history[-1][1] = gpt_response
    
    return "", chat_history  # Clear the text input and return updated history

# Create the Gradio interface with both audio input and chat
with gr.Blocks() as demo:
    gr.Markdown("# Audio Transcription Chatbot")
    gr.Markdown("Upload an audio file to transcribe and discuss, or type your message directly.")
    
    chatbot = gr.Chatbot(label="Conversation")
    
    with gr.Row():
        with gr.Column(scale=4):
            txt_input = gr.Textbox(
                show_label=False,
                placeholder="Type your message here...",
                container=False
            )
        with gr.Column(scale=1):
            audio_input = gr.Audio(
                type="filepath",
                label="Or upload audio"
            )
    
    # Setup event handlers
    txt_input.submit(submit_text, [txt_input, chatbot], [txt_input, chatbot])
    audio_input.change(process_audio_and_respond, [audio_input, chatbot], [chatbot])
    
    gr.Markdown("""
    ## How to use
    1. Upload an audio file to transcribe it and start a conversation about its content
    2. Or type a message directly to chat without audio
    3. The chatbot will remember the conversation context
    """)

# Launch the app
if __name__ == "__main__":
    demo.launch()

C:\Users\berna\AppData\Local\Temp\ipykernel_28396\690020612.py:81: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation")


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
